In [1]:
import spacy
from spacy.lang.en import English
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from bs4 import BeautifulSoup as BS
import glob
import time
from tqdm import tqdm
import re



In [2]:
The fields that you are looking to extract are as follows:

authorization_date: When was the repurchase authorized?
authorization_amount: Dollar amount of repurchases authorized
authorization_number: Number of repurchase shares authorized
repurchase_date: When did the repurchase occur?
repurchase_amount: Dollar amount of repurchases
repurchase_number: Number of shares repurchased

SyntaxError: invalid syntax (<ipython-input-2-681eb36db11c>, line 1)

In [3]:
with open('./data/mtn-20210731.htm', encoding='utf_8') as fi:
        fi = fi.read()
        soup = BS(fi, 'html.parser')

In [ ]:
for script in soup(["script", "style"]):
    script.decompose()



strips = list(soup.stripped_strings)
print(strips)

In [22]:
soup

<?xml version="1.0"?>
<html lang="en-US">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>mtn-20210731</title>
<style type="text/css">
 table.mtn20210731htm583 {border-collapse:collapse;display:inline-table;margin-bottom:5pt;vertical-align:text-bottom;width:94.005%}
 div.mtn20210731htm582 {margin-bottom:5pt;margin-top:5pt;text-align:center}
 td.mtn20210731htm581 {width:52.866%}
 td.mtn20210731htm580 {width:44.934%}
 table.mtn20210731htm579 {border-collapse:collapse;display:inline-table;margin-bottom:5pt;vertical-align:text-bottom;width:87.134%}
 td.mtn20210731htm578 {width:51.250%}
 td.mtn20210731htm577 {width:3.597%}
 td.mtn20210731htm576 {width:41.853%}
 td.mtn20210731htm575 {background-color:#cceeff;padding:2px 1pt;text-align:justify;vertical-align:top}
 td.mtn20210731htm574 {background-color:#ffffff;padding:2px 1pt;text-align:justify;vertical-align:top}
 div.mtn20210731htm573 {margin-bottom:5pt;margin-top:5pt;text-align:justify}
 div.mtn20210731ht

In [6]:
nlp = English()

In [7]:
doc = nlp(str(strips))
doc

['mtn-20210731', 'false2021FY000081201100008120112020-08-012021-07-31iso4217:USD00008120112021-01-31xbrli:shares00008120112021-09-2000008120112021-07-3100008120112020-07-31iso4217:USDxbrli:shares00008120112019-08-012020-07-3100008120112018-08-012019-07-310000812011us-gaap:CommonStockMember2018-07-310000812011mtn:ExchangeableSharesMember2018-07-310000812011us-gaap:AdditionalPaidInCapitalMember2018-07-310000812011us-gaap:AccumulatedOtherComprehensiveIncomeMember2018-07-310000812011us-gaap:RetainedEarningsMember2018-07-310000812011us-gaap:TreasuryStockMember2018-07-310000812011mtn:TotalVailResortsIncStockholdersEquityMember2018-07-310000812011us-gaap:NoncontrollingInterestMember2018-07-3100008120112018-07-310000812011us-gaap:RetainedEarningsMember2018-08-012019-07-310000812011mtn:TotalVailResortsIncStockholdersEquityMember2018-08-012019-07-310000812011us-gaap:NoncontrollingInterestMember2018-08-012019-07-310000812011us-gaap:AccumulatedOtherComprehensiveIncomeMember2018-08-012019-07-310000

In [9]:
nlp.max_length = 1500000


In [10]:
nlp = spacy.load('en_core_web_sm')

In [11]:
matcher = PhraseMatcher(nlp.vocab)

In [12]:
pattern = nlp(doc)
#matcher.add('DOG', None, pattern)

ValueError: [E109] Component 'parser' could not be run. Did you forget to call `initialize()`?

In [21]:
for match_id, start, end in matcher(doc):
# get the matched span
    span = doc[start:end]
    print('Matched span:', span.text)


In [22]:
month_patterns = [nlp("March") for doc in ("In March, I went on vacation")]


matcher = PhraseMatcher(nlp.vocab)
matcher.add('MONTH', None, *month_patterns)



In [23]:
test = nlp("March")
matches = matcher(test)
for match_id, start, end in matches:
    rule_id = nlp.vocab.strings[match_id]  # get the unicode ID, i.e. 'COLOR'
    span = doc[start : end]  # get the matched slice of the doc
    print(rule_id, span.text)


MONTH [


In [24]:
doc = nlp(str(sentences))
for ent in doc.ents:
    print(ent.text, ent.label_)


the\nfourth quarter of Fiscal 2021 DATE
2006 DATE
Company’s Board ORG
stock.\nOn July\xa016, 2008 PERSON
Company’s Board of Directors ORG
December 4, 2015 DATE
Company’s Board of Directors ORG
2021 DATE
Company ORG
6,161,141 CARDINAL
approximately $404.4\nmillion MONEY
July\xa031 DATE
2021 DATE
1,338,859 CARDINAL
Vail Shares ORG
Vail Holdings Credit Agreement ORG
Vail Shares ORG
Vail Shares ORG
These\nauthorizations CARDINAL
2006 DATE
Company’s Board of Directors ORG
Company ORG
3,000,000 CARDINAL
Vail Shares ORG
July\xa016, 2008 DATE
an additional\n3,000,000 Vail Shares ORG
December 4, 2015 DATE
Company ORG
1,500,000\nVail CARDINAL
Vail Shares ORG
during\nthe year ended July\xa031 DATE
2021 DATE
the year ended\nJuly\xa031, DATE
2020 DATE
Company ORG
256,418 CARDINAL
Vail Shares ORG
at\na ORG
$46.4 million MONEY
the year ended July\xa031,\n2019 DATE
Company ORG
353,007 CARDINAL
Vail Shares ORG
cost\nof $85.0 million MONEY
2021 DATE
Company ORG
approximately $404.4 million MONEY
of\nJul

In [30]:
sentences=[]

for sentence in strips:
    match=re.search(r"share repurchase program\, authorizing", sentence)
    if match: 
        sentences.append(sentence)


In [31]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(str(sentences))

expression= r"(share repurchase program\, authorizing )"
for match in re.finditer(expression, doc.text):
    start, end = match.span()
    span = doc.char_span(start, end)
    # This is a Span object or None if match doesn't map to valid token sequence
    if span is not None:
        print("Found match:", span.text)


In [36]:
sentences = re.sub(r"\\[a-z]+\d?", " ", str(sentences))
sentences

"['The Company did not repurchase any shares of common stock during the  quarter of Fiscal 2021. The share repurchase program is  under authorizations made from time to time by our Board  Directors. On March 9, 2006, the Company’s Board of Directors approved a share repurchase program, authorizing the Company to repurchase up to 3,000,000 shares of common stock. On July 16, 2008, the Company’s Board of Directors increased  authorization by an additional 3,000,000 shares, and  December 4, 2015, the Company’s Board of Directors increased the  by an additional 1,500,000 shares for a  authorization to repurchase shares of up  7,500,000 shares. Since inception of this stock  program through July 31, 2021, the Company has  6,161,141 shares at a cost of approximately $404.4 . As of July 31, 2021, 1,338,859 shares remained  to repurchase under the existing repurchase . Repurchases under these authorizations may be made  time to time at prevailing prices as permitted by applicable , and subject

In [35]:
doc = nlp(str(sentences))
for ent in doc.ents:
    print(ent.text, ent.label_)

the  quarter of Fiscal 2021 DATE
Board  Directors ORG
March 9, 2006 DATE
Company’s Board of Directors ORG
Company ORG
3,000,000 CARDINAL
July 16, 2008 DATE
Company’s Board of Directors ORG
an additional 3,000,000 CARDINAL
December 4, 2015 DATE
Company’s Board of Directors ORG
an additional 1,500,000 CARDINAL
7,500,000 CARDINAL
July 31, 2021 DATE
Company ORG
6,161,141 CARDINAL
approximately $404.4 MONEY
July 31, 2021 DATE
1,338,859 CARDINAL
Vail Shares ORG
Vail Holdings Credit Agreement ORG
Vail Shares ORG
Vail Shares ORG
March 9, 2006 DATE
Company’s Board of Directors ORG
Company ORG
3,000,000 CARDINAL
Vail Shares ORG
July 16, 2008 DATE
Company’s Board of Directors ORG
000,000 CARDINAL
Vail Shares ORG
December 4, 2015 DATE
Company’s Board  Directors ORG
1,500,000 CARDINAL
Vail Shares ORG
7,500,000 CARDINAL
Vail Shares ORG
2021 DATE
the year ended July 31, 2020 DATE
Company ORG
256,418 CARDINAL
Vail Shares ORG
$46.4 million MONEY
the year ended July 31 DATE
019 CARDINAL
Company ORG
353,